In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import os
path ="/content/drive/MyDrive/path/to/extract"
os.chdir(path)
os.listdir(path)

['Answers.csv',
 'Questions.csv',
 'Tags.csv',
 '.ipynb_checkpoints',
 'Questions_Clean.csv',
 'total.csv',
 'Total.csv',
 'stackoverflow_tags']

In [4]:
df1=open('Tags.csv', encoding="ISO-8859-1")
df_tags=pd.read_csv(df1)
df_tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
multi_label_tags_by_id = df_tags.groupby('Id')['Tag'].apply(list)

In [6]:
del df1

In [7]:
multi_label_tags_by_id.head()

Id
80                           [flex, actionscript-3, air]
90     [svn, tortoisesvn, branch, branching-and-merging]
120                              [sql, asp.net, sitemap]
180    [algorithm, language-agnostic, colors, color-s...
260         [c#, .net, scripting, compiler-construction]
Name: Tag, dtype: object

In [8]:
top_tags = df_tags['Tag'].value_counts().head(10).index.tolist()
tag_top10=df_tags[df_tags.Tag.isin(top_tags)]
print(tag_top10.shape)
tag_top10.head()
top_tags

(826739, 2)


['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [9]:
def filter_top_tags(tag_list):
  final_tag_list = None
  for tag in tag_list:
    if tag in top_tags:
      if final_tag_list:
        final_tag_list.append(tag)
      else:
        final_tag_list = [tag]

  return final_tag_list

In [11]:
filtered_tags_df = multi_label_tags_by_id.apply(filter_top_tags)

In [12]:
Question = pd.read_csv('/content/drive/MyDrive/path/to/extract/Questions_Clean.csv', encoding='iso-8859-1')
Question.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...
1,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...
2,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...
3,180,Function for creating color wheels,This is something I've pseudo-solved many time...
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...


In [13]:
total=pd.merge(Question, filtered_tags_df, on='Id').dropna()
print(total.shape)
total.head(25)

(706336, 4)


,Id,Title,Body,Tag
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,[c#]
5,330,Should I use nested classes in this case?,I am working on a collection of classes used f...,[c++]
8,650,Automatically update version number,I would like the version property of my applic...,[c#]
10,930,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,[c#]
11,1010,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,[c#]
12,1040,How do I delete a file which is locked by anot...,I'm looking for a way to delete a file which i...,[c#]
20,1760,.NET Unit Testing packages?,Getting back into a bit more .NET after a few-...,[c#]
24,2250,Datatable vs Dataset,I currently use a DataTable to get results fro...,[c#]
26,2530,How do you disable browser Autocomplete on web...,How do you disable autocomplete in the major b...,[html]
32,2780,Converting ARBG to RGB with alpha blending,Let's say that we have an ARGB color:\n\nColor...,[c#]


In [14]:
del filtered_tags_df, Question

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')
# from sklearn.metrics import roc_auc_score
# import tensorflow as tf
# from tensorflow.keras import models
# from tensorflow.keras import layers
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model

In [16]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(total['Tag'])
labels = multilabel_binarizer.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [17]:
multilabel_binarizer.fit_transform(total['Tag'])

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
train,test=train_test_split(total[:50000],test_size=0.25,random_state=24)

In [27]:
train.shape, test.shape

((37500, 4), (12500, 4))

In [28]:
X_train_t=train['Title']
X_train_b=train['Body']
y_train=multilabel_binarizer.transform(train['Tag'])
X_test_t=test['Title']
X_test_b=test['Body']
y_test=multilabel_binarizer.transform(test['Tag'])

In [29]:
import nltk
nltk.download('punkt')
sent_lens_t=[]
for sent in train['Title']:
    sent_lens_t.append(len(word_tokenize(sent)))
max(sent_lens_t)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


43

In [30]:
np.quantile(sent_lens_t,0.97)

19.0

In [31]:
max_len_t = 18
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
sequences_train_t = tok.texts_to_sequences(X_train_t)

In [32]:
vocab_len_t=len(tok.index_word.keys())
vocab_len_t

16052

In [33]:
from tensorflow.keras.preprocessing import sequence
sequences_matrix_train_t = sequence.pad_sequences(sequences_train_t,maxlen=max_len_t)
sequences_matrix_train_t

array([[   0,    0,    0, ..., 1304, 1132,  507],
       [   0,    0,    0, ...,   40,    2,  154],
       [   0,    0,    0, ..., 1556,    2,   36],
       ...,
       [   0,  277,  142, ...,    1,    5,  187],
       [   0,    0,    0, ...,    2,   16,   74],
       [   0,    0,    0, ..., 4746,  247,  186]], dtype=int32)

In [34]:
sequences_test_t = tok.texts_to_sequences(X_test_t)
sequences_matrix_test_t = sequence.pad_sequences(sequences_test_t,maxlen=max_len_t)

In [35]:
sequences_matrix_train_t.shape,sequences_matrix_test_t.shape,y_train.shape,y_test.shape

((37500, 18), (12500, 18), (37500, 10), (12500, 10))

In [36]:
sent_lens_b=[]
for sent in train['Body']:
    sent_lens_b.append(len(word_tokenize(sent)))
max(sent_lens_b)

7692

In [37]:
np.quantile(sent_lens_b,0.90)

387.0

In [38]:
max_len_b = 600
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_b)
sequences_train_b = tok.texts_to_sequences(X_train_b)

In [39]:
vocab_len_b =len(tok.index_word.keys())
vocab_len_b

140156

In [40]:
sequences_matrix_train_b = sequence.pad_sequences(sequences_train_b,maxlen=max_len_b)
sequences_matrix_train_b

array([[    0,     0,     0, ...,    14,    10,   172],
       [    0,     0,     0, ..., 13101,  3014,    89],
       [    0,     0,     0, ...,    55,     1,   136],
       ...,
       [    0,     0,     0, ...,    10,    89, 31326],
       [    0,     0,     0, ...,   114,    40,    10],
       [    0,     0,     0, ...,     3,    23,   219]], dtype=int32)

In [41]:
sequences_test_b = tok.texts_to_sequences(X_test_b)
sequences_matrix_test_b = sequence.pad_sequences(sequences_test_b,maxlen=max_len_b)

In [42]:
sequences_matrix_train_t.shape,sequences_matrix_train_b.shape,y_train.shape

((37500, 18), (37500, 600), (37500, 10))

In [43]:
sequences_matrix_test_t.shape,sequences_matrix_test_b.shape,y_test.shape

((12500, 18), (12500, 600), (12500, 10))

In [44]:


def RNN():
    # Title Only
    title_input = Input(name='title_input',shape=[max_len_t])
    title_Embed = Embedding(vocab_len_t+1,2000,input_length=max_len_t,mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(300)(title_Embed)
    # auxiliary output to tune GRU weights smoothly
    auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)

    # Body Only
    body_input = Input(name='body_input',shape=[max_len_b])
    body_Embed = Embedding(vocab_len_b+1,170,input_length=max_len_b,mask_zero=True,name='body_Embed')(body_input)
    gru_out_b = GRU(200)(body_Embed)

    # combined with GRU output
    com = concatenate([gru_out_t, gru_out_b])

    # now the combined data is being fed to dense layers
    dense1 = Dense(400,activation='relu')(com)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1)
    dense2 = Dense(150,activation='relu')(bn)

    main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)

    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    return model



In [45]:
model = RNN()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 title_input (InputLayer)       [(None, 18)]         0           []                               
                                                                                                  
 body_input (InputLayer)        [(None, 600)]        0           []                               
                                                                                                  
 title_Embed (Embedding)        (None, 18, 2000)     32106000    ['title_input[0][0]']            
                                                                                                  
 body_Embed (Embedding)         (None, 600, 170)     23826690    ['body_input[0][0]']             
                                                                                              

In [46]:


model.compile(optimizer='adam',loss={'main_output': 'categorical_crossentropy', 'aux_output': 'categorical_crossentropy'},
              metrics=['accuracy'])



In [47]:
model

In [48]:


results=model.fit({'title_input': sequences_matrix_train_t, 'body_input': sequences_matrix_train_b},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=3, batch_size=800)



Epoch 1/3
47/47 [==============================] - 138s 3s/step - loss: 3.8380 - main_output_loss: 1.8603 - aux_output_loss: 1.9777 - main_output_accuracy: 0.5160 - aux_output_accuracy: 0.4594 - val_loss: 3.1325 - val_main_output_loss: 1.7841 - val_aux_output_loss: 1.3484 - val_main_output_accuracy: 0.6265 - val_aux_output_accuracy: 0.6260
Epoch 2/3
47/47 [==============================] - 109s 2s/step - loss: 4.0214 - main_output_loss: 2.7603 - aux_output_loss: 1.2612 - main_output_accuracy: 0.6242 - aux_output_accuracy: 0.6865 - val_loss: 5.6781 - val_main_output_loss: 4.1166 - val_aux_output_loss: 1.5615 - val_main_output_accuracy: 0.5677 - val_aux_output_accuracy: 0.6494
Epoch 3/3
47/47 [==============================] - 106s 2s/step - loss: 6.8653 - main_output_loss: 5.4460 - aux_output_loss: 1.4193 - main_output_accuracy: 0.5050 - aux_output_accuracy: 0.6871 - val_loss: 7.3882 - val_main_output_loss: 5.4301 - val_aux_output_loss: 1.9581 - val_main_output_accuracy: 0.4039 - val_au

In [49]:


(predicted_main, predicted_aux)=model.predict({'title_input': sequences_matrix_test_t, 'body_input': sequences_matrix_test_b},verbose=1)



391/391 [==============================] - 52s 128ms/step


In [50]:


from sklearn.metrics import classification_report,f1_score



In [51]:


print(f1_score(y_test,predicted_main>.55,average='samples'))



0.2150927787767788


In [46]:
model.save('./stackoverflow_tags')

In [52]:
print(classification_report(y_test,predicted_main>.55))

              precision    recall  f1-score   support

           0       0.06      1.00      0.11       752
           1       0.25      1.00      0.40      3134
           2       0.10      1.00      0.18      1266
           3       0.07      0.93      0.13       756
           4       0.01      0.98      0.01        86
           5       0.17      1.00      0.29      2082
           6       0.12      1.00      0.22      1542
           7       0.45      0.88      0.60      1329
           8       0.14      1.00      0.25      1796
           9       0.09      1.00      0.17      1105

   micro avg       0.12      0.98      0.22     13848
   macro avg       0.15      0.98      0.24     13848
weighted avg       0.18      0.98      0.29     13848
 samples avg       0.12      0.99      0.22     13848



In [54]:
test.iloc[214]  #It's a multiclass multilabelled questions.

Id                                                  260210
Title    After clicking the nth item, how do I manipula...
Body     I'm using jQuery and wanting to target the nth...
Tag                                   [javascript, jquery]
Name: 3433, dtype: object

In [59]:
predicted_main[24].round(decimals = 2)

array([0.72, 0.61, 0.67, 0.61, 0.57, 0.74, 0.56, 0.37, 0.85, 0.66],
      dtype=float32)

In [60]:
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [61]:
model.save('./stackoverflow_tags_upadated')